In [1]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import matplotlib.pyplot as plt
import tensorflow.keras.utils as ku
import numpy as np
import pandas as pd

2023-07-05 10:43:21.228953: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-05 10:43:21.260224: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-05 10:43:21.806044: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Definition of a plot function for training result visualization

In [2]:
def plot_results(history):
    hist_df = pd.DataFrame(history.history)
    hist_df.columns = ["loss", "accuracy", "val_loss", "val_accuracy"]
    hist_df.index = np.arange(1, len(hist_df) + 1)

    fig, axs = plt.subplots(nrows=2, sharex=True, figsize=(16, 10))
    axs[0].plot(hist_df.val_accuracy, lw=3, label='Validation Accuracy')
    axs[0].plot(hist_df.accuracy, lw=3, label='Training Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].grid()
    axs[0].legend(loc=0)
    axs[1].plot(hist_df.val_loss, lw=3, label='Validation Loss')
    axs[1].plot(hist_df.loss, lw=3, label='Training Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].grid()
    axs[1].legend(loc=0)

    plt.show();

## Data preproccesing

In [3]:
# Mounting the google drive to google colab in order to load the data files directly from it
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


We load a txt file containing Shakespeare sonnets

In [3]:
data = open('sonnets.txt').read()

corpus = data.lower().split("\n")

In [4]:
corpus[0:10]

['from fairest creatures we desire increase,',
 "that thereby beauty's rose might never die,",
 'but as the riper should by time decease,',
 'his tender heir might bear his memory:',
 'but thou, contracted to thine own bright eyes,',
 "feed'st thy light'st flame with self-substantial fuel,",
 'making a famine where abundance lies,',
 'thyself thy foe, to thy sweet self too cruel.',
 "thou that art now the world's fresh ornament",
 'and only herald to the gaudy spring,']

We need to transform each text sentence into token sequence.

Then we need to generate from each token sequence, several token subsequences in order to augment the dataset. **Remember** that we want to learn how to predict the next word of a sentence. A sentence of 5 words can so be used to generate 4 training sequences.

e.g. the sentence "*to be or not to be*" can give the following training sequences: 

"*to* **be**"

"*to be* **or**"

"*to be or* **not**"

"*to be or not* **to**"

"*to be or not to* **be**"

In [6]:
tokenizer = Tokenizer()

# Question 1: use the preprocessing steps learned in course3_text_sequence_preprocessing_ex.ipynb to create the padded sequences needed to train the model
# Hint: Be careful about the length you will use for the padding sequences AND about where you put the extra zero coming from the padding (at the beginning of the sequence or at the end)

tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i + 1]
        input_sequences.append(n_gram_sequence)

max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

predictors, label = input_sequences[:, :-1], input_sequences[:, -1]

In [8]:
sequence_text = tokenizer.texts_to_sequences(['to be or not to be'])[0]
sequence_text

[3, 21, 36, 15, 3, 21]

## Neural network model definition

Build a neural network using at least one LSTM layer

(you may have a look at https://keras.io/api/layers/)

In [11]:

from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
import tensorflow as tf

# Question 2: define a neural network model using at least one LSTM layer
# Hint1: we advise you to use as first layer an Embedding layer (have a look at course3_sentiment_analysis_LSTM_ex.ipynb for a reminder of how to use it)
# Hint2: you can import any additional layers from tensorflow.keras.layers if needed

model = tf.keras.models.Sequential([
    Embedding(total_words, 100, input_length=max_sequence_len - 1),
    Bidirectional(LSTM(150, return_sequences=True)),
    Dropout(0.2),
    LSTM(100),
    Dense(total_words/2, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    Dense(total_words, activation='softmax')
])


# Question 3: define a relevant loss function and optimizer

loss_function = 'sparse_categorical_crossentropy'
optimizer = 'adam'

model.compile(loss=loss_function, optimizer=optimizer, metrics=['accuracy'])

print(model.summary())

2023-07-05 11:39:28.411669: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-05 11:39:28.412540: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-05 11:39:28.413228: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 10, 100)           321100    
                                                                 
 bidirectional_2 (Bidirectio  (None, 10, 300)          301200    
 nal)                                                            
                                                                 
 dropout_2 (Dropout)         (None, 10, 300)           0         
                                                                 
 lstm_5 (LSTM)               (None, 100)               160400    
                                                                 
 dense_2 (Dense)             (None, 1605)              162105    
                                                                 
 dense_3 (Dense)             (None, 3211)              5156866   
                                                      

2023-07-05 11:39:28.620999: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-05 11:39:28.621927: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-05 11:39:28.622558: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [ ]:
# Question 4: train your model with relevant parameters

predictors = predictors
label = label
epochs_value = 30
validation_split_value = ??????

history = model.fit(predictors, label, epochs=epochs_value, verbose=1, validation_split=validation_split_value)

## Result visualization

In [ ]:
plot_results(history)

## Text generation

You can now use your model to sequentially generate new words from a given uncomplete sentence

In [ ]:
seed_text = "Put here the begining of a sentence of your own"
nb_additional_words = 100

# Question 5: generate nb_additional_words at the end of seed_text according to your trained model
